### Done
* spacy lemmatisation don't remove proper names

### To Do
* train on more data??
* bigram trigram quad penta common phrases
* delete [music] and [laughter]

### Discussion and Findings
* Who is most popular? christian, dion, josh. no wait. deon? dionne?
* Most laughter in patch [Music] [Laughter]
* you you you you you bad transcription
* each patch (document) should be about a few topics
* each document consists of a mixture of topics
* each topic consists of a collection of words
* minimum word length 3 or 4 for dictionary? 'they'
* alpha per document topic
* common acrosss all samples: social_convention, daily_habit (read from script), now before we get started??
* now before we get starteddion
* dion, dione, dionne, deon
* what_did_you_sew

### Method
* acquire subtitles - screenshots
* supply source code on github notsal


### How it can be used
* segments find sound_check which patch

In [52]:
import pandas as pd
import en_core_web_sm
import gensim
import gensim.corpora as corpora
import nltk
import spacy
import sklearn
import gzip
import json

from nltk.corpus import stopwords
from gensim.utils import simple_preprocess, simple_tokenize
from gensim.models import CoherenceModel
from gensim.models.phrases import Phraser
from pprint import pprint

# Plotting tools
import matplotlib.pyplot as plt
#get_ipython().run_line_magic('matplotlib', 'inline')

import warnings
warnings.filterwarnings('ignore')

nlp = spacy.load("en_core_web_sm")

In [53]:
# functions

# read in subtitles and remove YouTube's auto-generated audio descriptions
def read_and_clean(patch_number):
    path = 'C:/Users/Sally/WTP/data/'
    open_patch = open(path+f"Patch {patch_number}_ At Home with Patchwork _ YouTube Live Stream - English (auto.txt").read()
    clean_patch = open_patch.replace('\n\n',' ').replace('[Music] ','').replace('[Laughter] ','')
    return clean_patch

# turn the strings into words
def sent_to_words(your_list):
    for item in your_list:
        #yield(gensim.utils.simple_preprocess(str(item),deacc=False,min_len=3))
        yield(gensim.utils.simple_preprocess(str(item),deacc=False,min_len=0))
        

# read in subtitles and remove YouTube's auto-generated audio descriptions
def read_and_clean_alt(filename):
    path = 'C:/Users/Sally/WTP/data/'
    open_patch = open(path+f"{filename}.txt").read()
    clean_patch = open_patch.replace('\n\n',' ').replace('[Music] ','').replace('[Laughter] ','')
    return clean_patch

In [54]:
p64 = read_and_clean(64)
p65 = read_and_clean(65)
p66 = read_and_clean(66)
p67 = read_and_clean(67)
p68 = read_and_clean(68)
p69 = read_and_clean(69)

p01 = read_and_clean_alt('At Home with Patchwork')
p02 = read_and_clean_alt('At Home with Patchwork _ Sat 4th April - English (auto')

all_patches = [p01,p02,p64,p65,p66,p67,p68,p69]

for z in all_patches:
    print(len(z))

66003
69581
32038
30748
35566
43389
38486
58425


In [55]:
#uses simple_preprocess
data_words = list(sent_to_words(all_patches))

In [56]:
for p in data_words:
    print(len(p))

13818
14639
6716
6486
7507
9082
8097
12298


### Pre-processing

In [57]:
# Define functions for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def make_quadgrams(texts):
    return [quadgram_mod[trigram_mod[doc]] for doc in texts]

In [58]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'PROPN']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def lemmatisation(texts):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

In [59]:
stop_words = stopwords.words('english')
more_sw = ['go','know','think','get','want']
stop_words.append(more_sw)

In [60]:
# build the bigram and trigram models

bigram = gensim.models.Phrases(data_words)
trigram = gensim.models.Phrases(bigram[data_words])
quadgram = gensim.models.Phrases(trigram[data_words])

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
quadgram_mod = gensim.models.phrases.Phraser(quadgram)

In [61]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [62]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Form trigrams
data_words_trigrams = make_trigrams(data_words_nostops) # this is a list

# Form trigrams
data_words_quadgrams = make_quadgrams(data_words_nostops) # this is a list

In [63]:
#for ngrams, _ in trigram.vocab.items():
for ngrams, _ in bigram.vocab.items():
    unicode_ngrams = ngrams.decode('utf-8')
    if '_' in unicode_ngrams:
        print(unicode_ngrams)

patch_workers
workers_welcome
welcome_it
it_s
s_great
great_to
to_see
see_dotted
dotted_christian
christian_i
i_can
can_see
see_organized
organized_but
but_it
see_that
that_the
the_chat
chat_coming
coming_through
through_welcome
welcome_to
to_our
our_second
second_youtube
youtube_live
live_show
show_christian
christian_down
down_how
how_are
are_you
you_how
you_going
going_dion
dion_very
very_good
good_josh
josh_i
i_never
never_would
would_have
have_thought
thought_you
you_d
d_ask
ask_a
a_question
question_like
like_that
that_can
can_we
we_do
do_something
something_more
more_original
original_in
in_our
our_lives
lives_and
and_just
just_asking
asking_hey
hey_how
how_you
you_doing
doing_we
we_can
can_show
show_you
you_than
than_that
that_what
what_a
a_boring
boring_thing
thing_that
that_has
has_happened
happened_to
to_you
you_in
in_the
the_last
last_two
two_years
years_i
i_had
had_a
a_meal
meal_last
last_night
night_it
s_a
a_japanese
japanese_meal
meal_it
it_had
had_garlic
garlic_in
in_it

chat_by
way_everyone
everyone_elias
elias_is
a_punctuation
punctuation_mark
mark_indicating
indicating_a
a_pause
pause_typically
typically_between
between_two
two_main
main_clauses
clauses_now
now_elliot
elliot_or
or_if
re_attempting
attempting_to
to_to
pass_that
that_as
as_your
your_own
own_definition
definition_that
a_copy
copy_and
and_paste
paste_job
job_right
right_there
there_i
m_sure
sure_so
so_might
might_just
just_go
go_back
back_to
the_semicolon
semicolon_for
a_moment
moment_so
think_the
using_that
that_because
because_not
not_many
many_people
people_know
know_how
how_to
use_it
correctly_you
you_got
be_damn
damn_sure
know_using
it_right
right_every
time_you
re_thinking
thinking_about
about_like
a_conjoined
conjoined_clause
clause_and
yeah_very
very_tip
tip_i
very_wary
wary_to
think_um
um_i
think_one
the_earmarks
earmarks_of
very_close
close_friendship
friendship_is
can_let
your_guard
guard_down
down_right
right_when
re_sending
sending_text
messages_so
have_probably
probably_ve

michael_and
the_picture
of_him
him_is
a_is
is_with
a_paintbrush
paintbrush_in
in_his
his_hand
reads_just
just_finished
finished_painting
painting_a
a_red
red_feature
feature_wall
wall_in
my_new
new_house
house_got
some_paint
paint_on
my_forehead
forehead_but
but_should
should_shop
shop_fine
fine_josh
you_were
were_scrolling
instagram_as
yeah_scotty
scotty_scotty
scotty_got
the_right
right_right
and_ended
the_count
count_of
a_marco
marco_polo
polo_at
at_silk
silk_road
road_warrior
warrior_and
the_image
image_he
he_had
had_was
a_table
table_covered
covered_in
in_piles
piles_of
of_beautiful
beautiful_colored
colored_spices
spices_and
caption_read
read_this
this_spice
spice_market
market_is
is_incredible
incredible_it
absolutely_massive
massive_i
i_keep
keep_losing
losing_my
my_friends
friends_though
though_they
they_keep
keep_shouting
shouting_my
my_first
first_name
name_and
i_respond
a_sermon
sermon_about
about_each
other_in
no_time
time_hashtag
hashtag_blind
blind_venetian
venetian_and


t_own
own_a
pair_but
ve_always
always_thought
ve_seen
seen_they
they_seem
seem_to
be_quite
quite_liberating
liberating_for
wear_them
look_really
comfy_it
s_id
id_on
you_find
find_particular
particular_offense
offense_about
about_people
people_where
m_shorts
shorts_over
over_like
a_place
place_to
discuss_patch
workers_unite
unite_do
do_men
men_need
wear_what
are_called
called_modesty
modesty_shorts
shorts_when
re_wearing
wearing_tights
tights_when
go_out
a_walk
walk_in
morning_you
you_pull
pull_on
of_tights
tights_do
wear_modesty
shorts_we
re_interested
interested_in
your_opinion
opinion_because
hate_having
wear_those
those_modesty
shorts_because
should_because
comfortable_just
just_wearing
wearing_leggings
leggings_it
my_one
one_opportunity
opportunity_and
and_sure
sure_how
how_tight
tight_are
your_modesty
shorts_they
not_tight
tight_but
got_like
this_sport
sport_shorts
shorts_and
the_mesh
mesh_you
christian_would
would_ordinarily
ordinarily_cut
cut_out
in_so
it_means
means_i
three_dif

the_home
home_happen
happen_like
m_sorry
just_excited
excited_so
so_because
person_out
out_even
a_light
light_hey
late_because
ve_not
not_already
already_departed
departed_no
i_disagree
disagree_with
got_another
another_online
online_call
other_group
group_of
of_podcasters
podcasters_that
a_podcast
podcast_with
problem_is
to_map
map_out
out_your
your_night
night_so
christian_if
we_started
started_at
at_i
call_at
to_push
push_that
that_call
call_out
it_kinda
kinda_has
has_ripple
ripple_effects
effects_on
on_sleep
sleep_and
and_rem
rem_sleep
and_cycles
cycles_and
and_everything
everything_but
i_knew
knew_i
knew_that
you_didn
minute_call
call_organised
organised_then
then_yeah
think_in
been_late
late_but
but_do
you_upset
an_excuse
excuse_a
good_excuse
excuse_would
been_good
good_like
have_liked
liked_a
excuse_like
a_stub
stub_a
a_toe
toe_on
the_floor
floor_of
the_kitchen
kitchen_got
got_stuck
stuck_up
the_toilet
toilet_sorry
sorry_it
really_bad
bad_traffic
traffic_all
the_stairs
stairs_go

...

windows_before
before_people
come_get
get_or
or_doors
doors_and
a_stiff
stiff_breeze
breeze_through
question_um
really_difficult
difficult_in
in_winter
winter_so
so_definitely
definitely_i
would_do
yeah_sometimes
sometimes_light
light_a
a_candle
candle_as
seem_too
too_staged
staged_do
doing_are
will_only
only_go
open_if
go_distinct
distinct_and
so_normally
normally_normally
fine_our
our_place
place_is
pretty_fine
fine_but
probably_crack
crack_a
window_just
bit_enough
makes_more
more_sense
sense_i
m_fine
fine_wallowing
wallowing_in
my_own
own_filth
filth_and
own_stench
stench_when
have_other
just_crack
window_a
over_as
huge_thing
re_hosting
hosting_a
party_there
more_effort
more_spread
spread_throughout
of_more
more_different
different_parts
house_yeah
yeah_to
saying_as
about_when
special_visitor
visitor_you
that_maybe
maybe_you
while_if
s_friends
friends_coming
over_sure
sure_like
quick_clean
clean_yeah
someone_special
special_that
re_preparing
preparing_the
the_platter
platter_beforeh

european_size
size_the
only_goes
by_serving
serving_sizes
sizes_are
are_an
absolute_joke
joke_one
one_serving
serving_of
of_shapes
shapes_art
art_its
its_shapes
shapes_is
get_seven
seven_you
seven_seven
seven_servings
servings_out
of_her
her_box
shapes_i
are_sittings
sittings_they
they_on
on_eric
eric_and
has_dropped
dropped_that
that_bone
with_shapes
shapes_in
single_comedic
comedic_endeavor
endeavor_he
s_done
done_across
across_the
the_board
board_for
for_anyone
anyone_he
called_game
boys_they
they_also
also_live
stream_he
re_obsessed
you_absolutely
absolutely_oh
this_your
your_supply
supply_shapes
shapes_have
have_what
you_looking
so_seven
seven_people
just_two
two_boxes
boxes_i
say_for
me_packets
packets_will
will_suit
suit_me
me_well
well_interesting
some_interesting
interesting_on
stream_just
episode_standard
standard_rule
rule_is
one_per
per_week
week_year
supply_cinema
cinema_tickets
tickets_is
is_years
years_supply
of_krispy
krispy_kreme
kreme_doughnuts
doughnuts_is
is_packs
p

been_hell
is_queues
queues_waiting
a_queue
queue_with
the_meters
meters_it
is_chaos
chaos_people
re_long
long_breyers
breyers_they
long_queues
queues_yeah
get_separated
separated_out
m_to
meet_super
super_the
day_was
was_strangers
ve_cut
cut_in
realize_when
i_first
first_slot
slot_slotted
slotted_in
realize_i
were_lining
lining_up
the_dots
dots_or
like_have
i_cut
just_when
when_my
my_turn
turn_came
came_i
i_turned
turned_to
said_are
go_great
to_sometimes
m_switched
would_appreciate
appreciate_if
how_a
stranger_becomes
when_good
good_strangers
strangers_become
become_good
friends_but
the_tipping
tipping_point
point_for
you_generally
me_sorry
a_smile
smile_i
think_ah
to_smile
smile_at
so_disarming
disarming_isn
smile_you
go_okay
friend_not
not_foe
foe_yeah
are_many
many_bad
bad_people
in_human
human_history
history_that
had_beautiful
beautiful_smiles
smiles_i
think_though
though_dion
smile_and
no_blinking
blinking_it
probably_going
to_tip
tip_you
edge_yeah
and_hot
hot_so
well_also
also_f

and_walk
fridge_you
be_closing
closing_the
fridge_why
the_reasoning
reasoning_for
shit_rule
rule_you
with_no
the_fridges
fridges_job
job_to
the_interior
interior_of
fridge_cool
cool_ryan
ryan_and
it_regulates
regulates_that
that_temperature
temperature_what
your_job
job_what
to_shut
shut_the
fuck_up
up_leave
fridge_okay
not_letting
letting_all
the_cold
air_escape
escape_and
work_harder
harder_to
cool_that
m_helping
helping_it
it_do
do_its
its_job
that_these
these_fridges
fridges_we
what_goes
into_them
re_sure
re_bloody
bloody_heavy
heavy_they
ask_fridges
fridges_this
make_fridges
fridges_are
so_heavy
heavy_you
move_a
big_fridge
fridge_they
are_impossible
move_so
made_they
they_blair
blair_makes
a_premium
premium_quality
quality_and
mean_josh
problem_if
you_went
to_antarctica
antarctica_i
getting_out
know_heating
up_heating
the_snowcaps
snowcaps_like
problem_here
here_sure
gonna_like
it_gonna
a_heater
heater_box
box_and
i_lived
lived_in
in_antarctica
antarctica_and
i_opened
opened_the
s

win_the
young_duke
duke_at
at_for
for_wins
wins_so
really_honed
honed_in
the_aesthetics
aesthetics_of
s_seriously
think_like
m_with
a_hardwood
floor_if
have_read
the_brief
brief_of
this_segment
segment_wrong
wrong_then
have_picked
picked_a
very_light
light_hardwood
of_aesthetics
aesthetics_i
s_comparable
comparable_with
with_concrete
concrete_slab
slab_for
me_big
big_bit
floor_just
of_tiptoeing
tiptoeing_on
floor_reminds
of_christmas
christmas_this
smell_the
cut_card
card_would
would_for
for_ever
ever_be
more_anticipation
anticipation_for
s_response
response_aren
just_interested
interested_because
obviously_not
not_hardwood
floor_or
or_grass
grass_are
just_only
just_hanging
off_but
honest_josh
go_anywhere
anywhere_near
near_carpet
carpet_i
am_going
to_tear
tear_you
to_pieces
pieces_i
gone_for
all_rounder
rounder_going
a_linoleum
linoleum_floor
floor_your
your_grandparents
grandparents_place
place_before
s_easy
your_spill
spill_cyst
cyst_back
back_ball
ball_on
floor_get
up_mop
mop_it
up

In [64]:
#for ngrams, _ in trigram.vocab.items():
for ngrams, _ in quadgram.vocab.items():
    unicode_ngrams = ngrams.decode('utf-8')
    if '_' in unicode_ngrams:
        print(unicode_ngrams)

patch_workers
workers_welcome
welcome_it
it_s
s_great
great_to
to_see
see_dotted
dotted_christian
christian_i
can_see
i_can_see
can_see_organized
organized_but
but_it
see_that
that_the
the_chat
chat_coming
coming_through
welcome_to
through_welcome_to
welcome_to_our
our_second
youtube_live
second_youtube_live
youtube_live_show
show_christian
christian_down
down_how
how_are
are_you
you_how
you_going
going_dion
dion_very
very_good
good_josh
josh_i
i_never
never_would
would_have
have_thought
thought_you
you_d
d_ask
ask_a
a_question
question_like
like_that
that_can
can_we
we_do
do_something
something_more
more_original
original_in
in_our
our_lives
lives_and
and_just
just_asking
asking_hey
hey_how
how_you
you_doing
doing_we
we_can
can_show
show_you
you_than
than_that
that_what
what_a
a_boring
boring_thing
thing_that
that_has
has_happened
happened_to
to_you
you_in
in_the
the_last
last_two
two_years
years_i
i_had
had_a
a_meal
meal_last
last_night
night_it
s_a
a_japanese
japanese_meal
meal_it
i

right_because
because_like
a_footnote
footnote_right
right_someone
someone_s
s_brigid
brigid_saying
saying_it
it_comes
comes_from
from_asterix
asterix_comes
from_references
references_yeah
so_dumb
dumb_so
so_maybe
maybe_now
now_the
the_next
next_level
level_of
of_that
is_we
we_need_to
need_to_provide
provide_sources
sources_and
and_references
references_what
what_about
that_why
why_can_t
can_t_why
can_t_we
a_footnotes
footnotes_built
built_into
into_whatsapp
whatsapp_what
that_right
right_you
you_go
oh_man
man_janet
janet_was
was_so
so_drunk
drunk_on
on_saturday
saturday_night
night_with
with_a_little
a_little_one
one_and
and_if
you_take
take_a
a_link
link_to
the_photo
photo_album
album_right
right_what
what_sources
sources_guys
guys_and
and_what
what_else
else_would
would_you
with_what
what_other
other_faces
faces_could
could_you
you_build
build_in
in_could
you_a
a_compare
compare_document
document_feature
feature_perhaps
perhaps_and
and_whatsapp
whatsapp_you
you_could
can_turn
turn_i

like_your
your_double
double_think
think_so
so_or
or_maybe
maybe_it
like_a_quick
a_quick_brown
brown_fox
fox_kind_of
kind_of_deal
deal_where
or_you
the_numbers
numbers_right
go_sorry
t_catch
catch_or
or_but
but_through
through_sounded
sounded_gray
gray_does
does_that
that_mean
mean_now
need_to_change
change_your
your_speech
speech_pattern
pattern_so
that_a
go_silent
silent_and_then
and_then_continue
continue_at
the_yeah
a_really
really_regular
regular_drop
out_you
just_report
report_okay
okay_josh
ll_tell
tell_you
you_about
about_my
my_thoughts
thoughts_on
on_fake
fake_meat
meat_my_favorite
thing_about
about_fake
meat_is
s_available
available_for
for_vegetarian
vegetarian_and
and_make
make_it
it_be
be_like
real_meat
meat_subsidy
subsidy_and
it_makes
makes_you
feel_good
good_well
consider_it
it_cold
cold_dropping
but_i_think
i_think_i
i_or
or_what
will_tend
do_though
though_i
i_probably
probably_will
will_jump
jump_straight
straight_to
the_text
message_but
but_again
again_it
s_normally


the_winner
winner_of
the_bug
bug_is
is_alanna
alanna_news
news_who
who_says
says_you
want_really_good
you_decide
decide_to
to_in
your_washing
washing_on
a_whim
whim_and
and_five_minutes
five_minutes_later
later_start
start_pouring
pouring_with
with_rain
rain_ah
ah_look
look_thank_you
for_listening
so_much_for_listening
for_listening_to
to_welcome_to
welcome_to_patchwork
patchwork_for
for_our
second_youtube
youtube_livestream
livestream_we_re
patch_in
a_minute
minute_but
do_we
we_we
we_get
into_our
get_into_our
into_our_patches
patches_and
yeah_thanks
thanks_so_much
for_joining
so_much_for_joining
for_joining_us
really_appreciate
us_really_appreciate
really_appreciate_it
it_we_ll
we_ll_have
have_another
another_half
half_hour
hour_of
patch_but
but_uh
uh_thirty
thirty_of
patch_did
what_patch_did
patch_did_you
sew_into
you_sew_into
sew_into_your
your_quilt
quilt_this_week
this_week_thank_you
very_much
thank_you_very_much
very_much_dionne
dionne_this_week
this_week_i
i_sewed
sewed_into
my_

in_any
any_amount
amount_of
money_go
and_then_once
once_the
person_organizing
organizing_it
gets_all
the_money
in_they
they_resize
resize_your
your_signature
signature_on
card_to_be
to_be_proportionate
proportionate_to
to_how_much
how_much_money
money_you
could_amazing
amazing_if
was_fun
fun_like
like_christian
christian_halfway
halfway_its
its_opposite
opposite_the
the_quarters
quarters_josh
yeah_alright
alright_well
we_ll_wrap
wrap_it
up_yeah
yeah_really
want_to_appreciate
appreciate_everyone
everyone_for_joining
us_today
today_it
been_a
a_wild
wild_ride
ride_just
to_remind
remind_if
t_here
here_at
we_are
are_actually
actually_going
to_be_live
streaming_all
our_podcasts
podcasts_for
the_foreseeable
foreseeable_future
future_thursday
nights_at
at_p_m
p_m_you
to_tune
to_us
us_it
ll_be_every
fortnight_and
and_of_course
of_course_they
they_ll_be
ll_be_recorded
recorded_and
know_in
your_podcast
podcast_apps
apps_and
and_remember
remember_if
patron_as_well
as_well_are
you_continue
continue

...

view_of
me_as
no_issues
issues_with
uh_loss
loss_of
of_hair
hair_i
go_as
as_bird
eye_as
like_um
um_obviously
obviously_my
camera_will
stay_firmly
firmly_ready
i_m_wearing
beanie_all_right
enough_pre
pre_chat
chat_uh
uh_let_s
to_too_much
too_much_trouble
trouble_preparing
preparing_your
house_before
a_visitor
visitor_comes
comes_around
around_if_questions
s_continue
through_life
life_unable
unable_to
to_accurately
accurately_convert
convert_fahrenheit
fahrenheit_to
to_celsius
celsius_grab_your
their_quilter
quilter_friendship
bachelor_and
patchwork_uh
up_uh
post_that
did_during
week_that
that_garnered
garnered_a
of_attention
attention_uh
was_based
based_off
off_upon
upon_our
our_classic
classic_one
classic_games
games_uh
uh_plate_or
plate_or_bowl
believe_was
dion_who
having_some
some_lasagna
lasagna_and
you_ate
ate_from
photo_we
question_was
was_asked
asked_plate_or
the_resounding
resounding_the
resounding_response
was_definitely
definitely_plate
with_plate
christian_so
just_kick
kick_t

took_ages
ages_they
t_track
track_it
it_obviously
obviously_didn
mean_much
much_to
myself_oh
difficult_time
time_at
moment_companies
companies_are
it_tough
them_know_what
know_what_just
happened_so
wanted_to_let
jacket_actually
actually_was
was_delivered
me_this
this_morning
morning_let_me
a_payment
payment_kind
ll_notice
notice_the
the_phrasing
phrasing_that
used_in
just_let_me
just_nice
and_light
light_let_me
payment_yeah
sentence_could
could_have_been
have_been_condensed
condensed_but
longer_you_re
was_lighter
lighter_yeah
more_palatable
palatable_yeah
yeah_much
much_lighter
lighter_much
this_incredible
incredible_down
jacket_that
ve_got_beside
beside_me
me_here
josh_uh
really_the
the_culmination
culmination_of
the_experience
experience_josh
josh_then
then_uh
asked_them
payment_and
did_they
say_josh
josh_hi
for_letting
letting_me
me_know
know_please
please_keep
order_and
refund_as
a_gesture
gesture_of
of_good
good_will
will_in
meantime_do
for_any_other
any_other_questions
questions_

burgess_ran
ran_away
away_to
the_circus
circus_maybe
maybe_john
circus_a
point_from
from_juilliard
juilliard_gretel
gretel_colleen
colleen_a
game_show
show_host
host_oh
point_we_ll
we_ll_make_sure
can_afford
afford_is
show_game
show_need
need_at
at_wheel
wheel_anyway
anyway_i_m
sorry_wait
second_wait
second_if
can_win
a_million
million_dollars
dollars_playing
playing_of
the_reality
reality_show
show_i_think
becomes_a
show_all_right
all_right_we_ll
with_julia
julia_offline
offline_um
um_fellas
fellas_always
always_i
the_traffic
traffic_lights
lights_the
other_day_it
was_pouring
pouring_and
an_umbrella
had_an_umbrella
an_umbrella_in
hand_had
this_girl
girl_came
came_up
been_raining
raining_it
actually_wasn
t_pouring
pouring_it
light_sort_of
sort_of_mist
mist_between
light_mist
mist_and
poor_anyway
anyway_and
up_beside
raining_all
all_morning
morning_in
fact_to_be
to_be_raining
my_thought
was_hang_on
hang_on_she
she_knew
knew_when
she_left
left_this
morning_that
was_raining
raining_and
my

s_being
being_used_to
used_to_clean
get_dirty
dirty_in
kitchen_i
getting_dirty
dirty_i
d_be_used
up_fashion
fashion_i_think
the_tux
tux_is
can_squeeze
squeeze_out
of_chucks
chucks_and
s_practically
practically_dry
dry_it
ultimately_if
for_cleaning
it_contaminates
contaminates_when
know_drying
drying_the
contaminates_that
that_process
process_what
does_have
have_is
you_expect
expect_josh
josh_dries
dries_dishes
dishes_now
now_we
t_dry
dry_the
dishes_though
though_why
why_drive
drive_it
we_dry
the_rack
rack_they
big_pot
pot_that
t_come
you_trying
trying_that
right_away
tea_towel_might_be
might_be_the
the_so
beneath_the
spills_i_m
talking_about_capsicum
capsicum_seeds
seeds_i_m
talking_about_off
off_cuts
cuts_of
onion_yep
yep_and_then
board_you
the_four
four_corners
corners_together
together_and_then
you_run
run_away_from
away_from_home
home_so
other_day_after
after_about
about_good
good_long
long_years
and_hassling
hassling_of
of_my_girlfriend
my_girlfriend_i
to_finally
finally_clean
cle

wet_like
like_non
non_like
like_waterproof
waterproof_material
material_i_think
biggest_issue
wearing_short
sleeve_things
things_underneath
underneath_so
so_full
full_exposure
exposure_full
exposure_and
s_force
force_fed
fed_that
us_like
like_children
children_and
i_guess_that
means_we_ll
we_ll_segue
segue_into
into_a_little
segment_we
call_nursery
nursery_rhymes
rhymes_we
done_this
very_long_time
long_time_basically
basically_what
s_involved
involved_with
have_each
each_gone
gone_away
away_made
made_up
a_nursery
nursery_rhyme
rhyme_and
we_ve_also
also_found
found_a
rhyme_that
is_real
real_what
is_read
read_each_other
each_other_the
the_fake
fake_one
to_guess
guess_which_is
which_is_real
real_or
or_fake
fake_josh
josh_may
may_i
you_lead
lead_this
long_time_it
really_one
my_favorite_favorite
favorite_segments
segments_the
big_bumper
episode_we_ve
we_ve_included
included_this
in_i_m
really_looking
looking_forward
this_we_ve
we_ve_prepared
prepared_one
is_nonsensical
nonsensical_and
real_

In [65]:
%%time

data_lemmatised = lemmatisation(data_words_quadgrams)

Wall time: 3.26 s


In [66]:
#for i in quadgram.vocab.items():
#    print(i)

In [67]:
doc = 5
# print(all_patches[doc])
# print(data_words[doc])
# print(data_lemmatised[doc])
# print(data_words_nostops[doc])
# print(data_words_bigrams[doc])
# print(data_words_trigrams[doc])
print(data_words_quadgrams[doc])
# print('\n')

['hello', 'patchworkers', 'good', 'see', 'camera', 'good', 'see', 'christian', 'good', 'see', 'dion', 'christian', 'good', 'see', 'josh', 'okay', 'little', 'flush', 'bit', 'red', 'wine', 'sure', 'going', 'half', 'glass', 'feel_like', 'skin', 'cracking', 'oh', 'christian', 'sort', 'little', 'bit', 'uh', 'little', 'bit', 'horny', 'little', 'yeah', 'good', 'sign', 'christian', 'hoardy', 'skin', 'goes', 'dry', 'well', 'moisture', 'going', 'somewhere', 'else', 'guys', 'oh', 'god', 'cracking', 'must', 'ready', 'go', 'think', 'guys', 'make', 'episode', 'uh', 'good', 'never', 'described', 'flash', 'though', 'described', 'deck', 'cards', 'flush', 'never', 'uh', 'never', 'christian', 'flush', 'usually', 'relate', 'look', 'feel', 'well', 'probably', 'little', 'bit', 'feel', 'moment', 'look', 'flush', 'feel', 'shortly', 'success', 'well', 'uh', 'thank', 'everyone', 'uh', 'youtube_live', 'joining', 'us', 'today', 'get', 'cracking', 'episode', 'uh', 'shortly', 'um', 'bloody', 'cold', 'melbourne', 't

## Dictionary

In [68]:
id2word_raw = corpora.Dictionary(data_words)
id2word_raw.save_as_text('dictionary_raw.txt',sort_by_word=False)
print(len(id2word_raw))

id2word = corpora.Dictionary(data_lemmatised)
id2word.save_as_text('dictionary.txt',sort_by_word=False)
print(len(id2word))

5222
4199


In [69]:
d_word = [id2word[i] for i in id2word]
d_cf = [id2word.cfs[i] for i in id2word]
d_df = [id2word.dfs[i] for i in id2word]

bigdict = pd.DataFrame(zip(d_word,d_cf,d_df),columns=['word','count','patch'])
dict = pd.DataFrame(zip(d_word,d_cf,d_df),columns=['word','count','patch'])
dict = dict[dict['count']>1]
dict = dict.sample(n=100,random_state=4242)

In [70]:
dict

,word,count,patch
2155,ship,6,5
588,handle,10,4
3326,immediately,3,2
36,alone,5,5
1226,socket,3,1
...,...,...,...
4145,stunning,2,1
2038,plant,5,3
1259,start,71,8
84,audio,2,2


In [71]:
bigdict

,word,count,patch
0,-PRON-,128,8
1,aaron,2,1
2,able,26,8
3,absolute,10,4
4,absolutely,51,8
...,...,...,...
4194,woolen,2,1
4195,wording,1,1
4196,yell,1,1
4197,yesterday,1,1


In [72]:
import plotly.express as px
df = dict

#fig = px.scatter(df, x="count", y="patch", text="word", log_x=False, size_max=20, color="word")
fig = px.scatter(df, x="count", y="patch", text="word", log_x=True, size_max=20, color="count")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Patchword Vocab', title_x=1)
fig.update_layout(autosize=False,width=1200,height=500)
fig.show()

In [73]:
dict

,word,count,patch
2155,ship,6,5
588,handle,10,4
3326,immediately,3,2
36,alone,5,5
1226,socket,3,1
...,...,...,...
4145,stunning,2,1
2038,plant,5,3
1259,start,71,8
84,audio,2,2


In [74]:
bigdict

,word,count,patch
0,-PRON-,128,8
1,aaron,2,1
2,able,26,8
3,absolute,10,4
4,absolutely,51,8
...,...,...,...
4194,woolen,2,1
4195,wording,1,1
4196,yell,1,1
4197,yesterday,1,1


In [75]:
w = dict['word']

for i, txt in enumerate(w):
    print(str(txt))

ship
handle
immediately
alone
socket
feedback
exclusively
wine
period
speaker
average
mr
gretl
era
awful
amazing
desk
loud
scrolling
caption
zoom
used
soup
shocking
spell
vodka
presume
current
dinner
meet
balance
paper
perkin
realize
attitude
not
isolation
ahead
specific
movement
roast
pink
rule
favorite
cartoon
kitchen
manual
traditional
towel
tiny
concerned
refrie
number
picture
video_call
slow
victorian
chinese
drunk
roy
combat
corner
phone
flush
commas
bad
checking
betty
royal
jasmine
scour
switch
post
foil
fold
issue
contact
oh
expensive
michael
ingredient
squish
far
pant
fork
model
edit
disagree
employee
fight
dolly
fool
worried
crazy
people
stunning
plant
start
audio
famous


In [25]:
len(w)

100

In [26]:
type(w)

pandas.core.series.Series

In [27]:
w

3619          gorgeous
240           computer
752              light
2707    congratulation
1464               wow
             ...      
865               news
329          different
105          basically
2880         sacrifice
443            excited
Name: word, Length: 100, dtype: object

In [86]:
bigdict.to_csv('dictionary.csv')

In [83]:
# The word butter is mentioned 9 times across 4 patches ??????

#for i in range(1,3000,100):
for i in range(10,4000,100):
    print('The word',id2word[i],'is mentioned',id2word.cfs[i],'times across',id2word.dfs[i],'patches')

The word action is mentioned 2 times across 2 patches
The word bay is mentioned 1 times across 1 patches
The word clean is mentioned 32 times across 4 patches
The word degree is mentioned 4 times across 4 patches
The word episode is mentioned 36 times across 8 patches
The word forty is mentioned 1 times across 1 patches
The word hi is mentioned 35 times across 6 patches
The word kid is mentioned 10 times across 6 patches
The word meat is mentioned 5 times across 2 patches
The word organization is mentioned 1 times across 1 patches
The word pretty_much is mentioned 6 times across 3 patches
The word rightly is mentioned 2 times across 1 patches
The word size is mentioned 23 times across 5 patches
The word temperature is mentioned 6 times across 5 patches
The word viper is mentioned 1 times across 1 patches
The word animal is mentioned 4 times across 3 patches
The word chunky is mentioned 1 times across 1 patches
The word disclosure is mentioned 1 times across 1 patches
The word fuck is m

In [32]:
type(id2word)

gensim.corpora.dictionary.Dictionary

In [33]:
id2word.cfs

{942: 101,
 1460: 3,
 1439: 48,
 571: 141,
 1152: 183,
 357: 1,
 202: 233,
 912: 1,
 195: 81,
 228: 133,
 1151: 39,
 1483: 25,
 1186: 88,
 553: 718,
 332: 93,
 558: 188,
 694: 326,
 863: 57,
 1462: 152,
 1321: 482,
 74: 79,
 1038: 69,
 753: 947,
 1231: 154,
 913: 8,
 749: 44,
 609: 60,
 142: 2,
 1320: 263,
 592: 67,
 723: 46,
 1371: 79,
 1471: 26,
 808: 21,
 870: 37,
 682: 3,
 533: 3,
 493: 1,
 791: 3,
 327: 5,
 1190: 11,
 1282: 5,
 247: 15,
 435: 40,
 717: 525,
 431: 14,
 1101: 13,
 501: 10,
 425: 11,
 1048: 1,
 182: 1,
 59: 23,
 579: 142,
 545: 85,
 1128: 31,
 1346: 28,
 1420: 264,
 1138: 289,
 926: 1,
 736: 1,
 790: 37,
 364: 28,
 1109: 271,
 764: 59,
 1428: 54,
 95: 164,
 1265: 28,
 234: 24,
 775: 86,
 196: 2,
 882: 12,
 251: 6,
 1383: 40,
 832: 30,
 539: 745,
 442: 4,
 862: 4,
 311: 1,
 1131: 2,
 51: 3,
 522: 16,
 567: 1,
 443: 7,
 444: 4,
 116: 4,
 1228: 1,
 549: 6,
 1474: 86,
 144: 6,
 1429: 33,
 40: 72,
 1299: 40,
 1302: 3,
 861: 103,
 1092: 1,
 1283: 47,
 1207: 18,
 329: 46,
 

In [87]:
patch_dict = pd.read_csv('dictionary.txt',names=['id','word','freq'],sep='\s+')

In [88]:
patch_dict.sort_values(by='freq',ascending=False).head(5)

,id,word,freq
1,942,patch,8.0
127,1229,someone,8.0
118,1232,sometimes,8.0
119,771,long,8.0
120,72,around,8.0


In [89]:
popular_words = patch_dict[patch_dict.freq < 2]
popular_words.head(5)

,id,word,freq
1766,357,dotted,1.0
1767,912,organized,1.0
1768,493,fog,1.0
1769,1048,ramification,1.0
1770,182,catastrophic,1.0


In [90]:
print(id2word)

Dictionary(4199 unique tokens: ['-PRON-', 'aaron', 'able', 'absolute', 'absolutely']...)


### Removing rubbish from corpus

In [91]:
print([k for k,v in id2word.items() if v=='know'])
print([k for k,v in id2word.items() if v=='think'])
print([k for k,v in id2word.items() if v=='hello'])

[717]
[1321]
[607]


In [92]:
del_ids = 433,818,365

In [93]:
# remove unwanted word ids from the dictionary in place
id2word.filter_tokens(bad_ids=del_ids)

In [94]:
len(id2word)

4196

In [95]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_lemmatised]

## Topic Modelling

In [96]:
import os
mallet_path = "C:/Users/Sally/WTP/mallet-2.0.8/bin/mallet"
os.environ['MALLET_HOME'] = 'C:/Users/Sally/WTP/mallet-2.0.8'

In [100]:
num_topics = 30
num_words = 10

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path
                                            ,corpus=corpus
                                            ,num_topics=num_topics
                                            ,id2word=id2word
                                             )
pprint(ldamallet.show_topics(num_topics,num_words,formatted=False))

[(0,
  [('back', 0.08048289738430583),
   ('message', 0.0744466800804829),
   ('power_board', 0.04627766599597585),
   ('call', 0.03822937625754527),
   ('shoe', 0.03219315895372234),
   ('plug', 0.028169014084507043),
   ('short', 0.02414486921529175),
   ('power', 0.02012072434607646),
   ('phone_call', 0.018108651911468814),
   ('charge', 0.018108651911468814)]),
 (1,
  [('bowl', 0.054466230936819175),
   ('lolly', 0.0457516339869281),
   ('bender', 0.037037037037037035),
   ('wrap', 0.030501089324618737),
   ('fridge', 0.026143790849673203),
   ('tupperware', 0.023965141612200435),
   ('day', 0.023965141612200435),
   ('leftover', 0.023965141612200435),
   ('mentos', 0.017429193899782137),
   ('sock', 0.017429193899782137)]),
 (2,
  [('josh', 0.09345479082321187),
   ('christian', 0.06443994601889338),
   ('good', 0.06342780026990553),
   ('yeah', 0.058029689608636977),
   ('bit', 0.050944669365722),
   ('big', 0.03913630229419703),
   ('-PRON-', 0.037112010796221326),
   ('back', 

In [101]:
# num_topics = 8
# num_words = 8

# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path
#                                             ,corpus=corpus
#                                             ,num_topics=num_topics
#                                             ,id2word=id2word
#                                              )
# pprint(ldamallet.show_topics(num_topics,num_words,formatted=False))

In [102]:
def explore_topic(topic_number, topn=25):        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
    for term, frequency in ldamallet.show_topic(topic_number, topn=25):
        print(term,frequency)

In [103]:
explore_topic(topic_number=1)

term                 frequency

bowl 0.054466230936819175
lolly 0.0457516339869281
bender 0.037037037037037035
wrap 0.030501089324618737
fridge 0.026143790849673203
leftover 0.023965141612200435
tupperware 0.023965141612200435
day 0.023965141612200435
sock 0.017429193899782137
mentos 0.017429193899782137
peg 0.015250544662309368
foil 0.015250544662309368
sample 0.015250544662309368
mint 0.015250544662309368
heat 0.013071895424836602
free 0.013071895424836602
celebration 0.013071895424836602
reception_desk 0.013071895424836602
premium 0.013071895424836602
long 0.010893246187363835
mento 0.010893246187363835
reheat 0.010893246187363835
glad 0.010893246187363835
every_single 0.010893246187363835
container 0.010893246187363835


In [112]:
wtf = gensim.models.ldamodel.LdaModel(corpus=corpus
                                            ,id2word=id2word
                                            ,num_topics=10
                                            £,random_state=100
                                            #update_every=1,
                                            #chunksize=10,
                                            £,passes=10
                                            #alpha='symmetric',                                            
                                            £,alpha=0.0
                                            £,iterations=30,
                                            #per_word_topics=False
                                     )
#wtf.show_topics(num_topics,num_words,formatted=False)

c:\users\sally\appdata\local\programs\python\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



SyntaxError: invalid character in identifier (<ipython-input-112-4c5a42b00ba3>, line 4)

In [110]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(ldamallet, corpus,dictionary=id2word)
vis = pyLDAvis.gensim.prepare(wtf, corpus,dictionary=id2word)
vis

c:\users\sally\appdata\local\programs\python\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.127320  0.011932       1        1  46.484793
8      0.148713  0.046155       2        1  25.281055
5      0.146585  0.046296       3        1  19.931923
1      0.124941 -0.114557       4        1   8.287408
2     -0.056896  0.000192       5        1   0.002470
9     -0.062563  0.004849       6        1   0.002470
6     -0.104205  0.001927       7        1   0.002470
4     -0.104005  0.001058       8        1   0.002470
3     -0.107602  0.002115       9        1   0.002470
7     -0.112288  0.000034      10        1   0.002470, topic_info=       Term        Freq       Total Category  logprob  loglift
751    like  902.000000  902.000000  Default  30.0000  30.0000
537     get  710.000000  710.000000  Default  29.0000  29.0000
551      go  682.000000  682.000000  Default  28.0000  28.0000
1467   yeah  546.000000  546.000000  Default  27.0000  27.0000
715    know  500.000000  500.000000  Default  26.0000  26.0000
...     ...         ...         ...      ...      ...      ...
939   patch    0.000339   96.803383  Topic10  -7.7669  -1.9541
1337   time    0.000358  154.758937  Topic10  -7.7117  -2.3680
858    need    0.000335   98.939099  Topic10  -7.7772  -1.9862
556    good    0.000362  179.093081  Topic10  -7.7019  -2.5043
577     guy    0.000337  135.945791  Topic10  -7.7710  -2.2977

[772 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         1  0.491032       -PRON-
0         2  0.294619       -PRON-
0         3  0.147310       -PRON-
0         4  0.065471       -PRON-
1489      1  0.739320         ache
...     ...       ...          ...
3779      1  1.012686  year_supply
2647      4  0.782165           yo
2649      4  0.917272          zip
2650      4  0.782637       zipper
2651      4  0.783037         zips

[700 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 9, 6, 2, 3, 10, 7, 5, 4, 8])

In [258]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatised, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ',coherence_lda)

c:\users\sally\appdata\local\programs\python\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.23558925484608992


### bigrams and trigrams

In [149]:
from gensim.models import word2vec, Phrases
documents = all_patches
sentence_stream = [doc.split(" ") for doc in documents]

bigram = Phrases(sentence_stream, min_count=1000, threshold=20,delimiter=b' ')

INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:collected 28973 word types from a corpus of 47340 words (unigram + bigrams) and 6 sentences
INFO:gensim.models.phrases:using 28973 counts as vocab in Phrases<0 vocab, min_count=1000, threshold=20, max_vocab_size=40000000>


In [150]:
for j in bigram.vocab.items():        
    print(j)

(b'you', 1368)
(b'hello', 18)
(b'you hello', 2)
(b'and', 1061)
(b'hello and', 1)
(b'welcome', 33)
(b'and welcome', 2)
(b'to', 1088)
(b'welcome to', 31)
(b'patch', 60)
(b'to patch', 5)
(b'64', 2)
(b'patch 64', 1)
(b'what', 436)
(b'64 what', 1)
(b'a', 1391)
(b'what a', 10)
(b'disaster', 1)
(b'a disaster', 1)
(b"it's", 613)
(b"disaster it's", 1)
(b'vain', 1)
(b"it's vain", 1)
(b"let's", 46)
(b"vain let's", 1)
(b"let's let's", 3)
(b'talk', 22)
(b"let's talk", 3)
(b'about', 235)
(b'talk about', 16)
(b'internet', 3)
(b'about internet', 1)
(b'providers', 1)
(b'internet providers', 1)
(b'providers and', 1)
(b'super', 3)
(b'and super', 1)
(b'loop', 1)
(b'super loop', 1)
(b'in', 551)
(b'loop in', 1)
(b'particular', 5)
(b'in particular', 1)
(b"that's", 252)
(b"particular that's", 1)
(b'not', 201)
(b"that's not", 12)
(b'it', 825)
(b'not it', 1)
(b'is', 530)
(b'it is', 41)
(b"is let's", 2)
(b'D', 4)
(b'about D', 1)
(b'on', 392)
(b'D on', 2)
(b'the', 1716)
(b'on the', 84)
(b'fact', 20)
(b'the fact',

(b'order', 17)
(b'is order', 1)
(b'order of', 2)
(b'operations', 2)
(b'of operations', 2)
(b'operations I', 1)
(b'I need', 4)
(b'get that', 7)
(b'that order', 1)
(b'operations right', 1)
(b'right in', 2)
(b'the most', 19)
(b'most efficient', 1)
(b'efficient way', 1)
(b'way possible', 1)
(b'possible and', 1)
(b'and my', 7)
(b'favorite', 32)
(b'my favorite', 15)
(b'favorite and', 2)
(b"and I've", 7)
(b'killed', 1)
(b"I've killed", 1)
(b'killed a', 1)
(b'of time', 2)
(b'time thinking', 2)
(b'about this', 13)
(b'is in', 6)
(b'breakfast', 7)
(b'the breakfast', 1)
(b'routine', 3)
(b'breakfast routine', 1)
(b'between', 10)
(b'routine between', 1)
(b'cereal', 2)
(b'between cereal', 1)
(b'milk', 7)
(b'cereal milk', 1)
(b'Bowl', 2)
(b'milk Bowl', 1)
(b'toast', 8)
(b'Bowl toast', 1)
(b'fridge', 48)
(b'toast fridge', 1)
(b'fridge those', 1)
(b'those kind', 2)
(b'of things', 6)
(b'things so', 2)
(b"I'm thinking", 1)
(b'thinking right', 1)
(b"right I'm", 2)
(b"I'm like", 5)
(b'like okay', 1)
(b'so m

(b'wings beneath', 1)
(b'L', 2)
(b'beneath L', 1)
(b'L we', 1)
(b'we say', 1)
(b'say the', 4)
(b'bola', 1)
(b'the bola', 1)
(b'proverbial', 1)
(b'bola proverbial', 1)
(b'proverbial wing', 1)
(b'guns', 1)
(b'wing guns', 1)
(b'guns and', 1)
(b'yeah we', 5)
(b'love you', 2)
(b'you we', 2)
(b'this all', 5)
(b'all for', 1)
(b'free and', 2)
(b'and patrons', 1)
(b'patrons is', 1)
(b'can sort', 2)
(b'sustain', 1)
(b'of sustain', 1)
(b'sustain this', 1)
(b'this so', 3)
(b'patreon.com', 4)
(b'a patreon.com', 1)
(b'patreon.com forward', 2)
(b'bak', 1)
(b'slash bak', 1)
(b'bak and', 1)
(b'patchwork do', 1)
(b'it now', 5)
(b"now it's", 12)
(b'lovely', 13)
(b'a lovely', 3)
(b'lovely thing', 1)
(b'thing to', 4)
(b'but as', 6)
(b'do every', 7)
(b'every week', 7)
(b'week we', 9)
(b'we sew', 2)
(b'patch into', 8)
(b'into our', 9)
(b'quilt', 8)
(b'our quilt', 4)
(b'quilt of', 3)
(b'of friendship', 4)
(b'friendship Josh', 4)
(b'Josh what', 5)
(b'what patch', 11)
(b'patch did', 12)
(b'did you', 42)
(b'you 

(b'or five', 1)
(b'five steps', 1)
(b'steps it', 1)
(b'goes I', 1)
(b'I gotta', 1)
(b'make the', 2)
(b'he comes', 1)
(b'comes back', 1)
(b'and answer', 1)
(b'answer like', 1)
(b'like two', 1)
(b'two or', 2)
(b'or three', 2)
(b'three more', 1)
(b'more just', 3)
(b'just like', 13)
(b'just load', 1)
(b'sir', 3)
(b'yes sir', 1)
(b'sir so', 1)
(b'reckon Josh', 1)
(b'to ask', 12)
(b'ask you', 12)
(b'ask for', 2)
(b'mint or', 1)
(b'just take', 3)
(b'offer', 2)
(b'off offer', 1)
(b'offer of', 1)
(b'desk you', 1)
(b'could just', 3)
(b'take absolutely', 1)
(b'absolutely just', 1)
(b'just taking', 1)
(b'taking what', 1)
(b'Chris', 1)
(b'what Chris', 1)
(b"Chris you're", 1)
(b"you're asking", 1)
(b'asking I', 1)
(b'my boy', 1)
(b'boy I', 1)
(b'I understand', 1)
(b"understand it's", 1)
(b'assumed', 1)
(b"it's assumed", 1)
(b'assumed that', 1)
(b'you still', 3)
(b'still need', 1)
(b'ask no', 1)
(b"don't reckon", 1)
(b'reckon you', 4)
(b"that's just", 4)
(b"just it's", 5)
(b'overkill', 2)
(b'just ove

(b'recreating what', 1)
(b"what i've", 3)
(b'got set', 1)
(b'up here', 1)
(b'i want', 4)
(b'see the', 3)
(b'enjoyment', 1)
(b'the enjoyment', 1)
(b'enjoyment across', 1)
(b'across your', 1)
(b'face all', 1)
(b'right okay', 1)
(b'that yeah', 3)
(b'yeah uh', 2)
(b'uh now', 2)
(b'now here', 1)
(b'here at', 1)
(b'patchwork every', 1)
(b'every now', 2)
(b'and again', 2)
(b'again we', 1)
(b'help', 7)
(b'uh help', 1)
(b'help you', 3)
(b'learn', 4)
(b'guys learn', 1)
(b'learn something', 1)
(b'increase', 2)
(b'and increase', 1)
(b'increase your', 1)
(b'vocabulary', 1)
(b'your vocabulary', 1)
(b'vocabulary so', 1)
(b'patchword', 1)
(b'for patchword', 1)
(b'patchword of', 1)
(b'day christian', 1)
(b'christian the', 3)
(b'the policeman', 1)
(b'policeman started', 1)
(b'started to', 1)
(b'become anxious', 1)
(b'anxious what', 1)
(b'originally', 1)
(b'was originally', 1)
(b'originally one', 1)
(b'member', 1)
(b'one member', 1)
(b'member of', 1)
(b'kukaracha', 1)
(b'the kukaracha', 1)
(b'motorcycle'

...

(b"it's hit", 1)
(b'hit eight', 1)
(b"o'clock you", 1)
(b'just sort', 2)
(b'of hanging', 1)
(b'hanging around', 1)
(b'around doing', 1)
(b'doing nothing', 1)
(b'nothing yeah', 1)
(b'unlock', 1)
(b'you unlock', 1)
(b'unlock the', 1)
(b"it's shocking", 1)
(b"shocking isn't", 1)
(b"you're ready", 1)
(b'do anything', 1)
(b"can't start", 1)
(b'start something', 1)
(b'else because', 1)
(b'mood has', 1)
(b'has settled', 1)
(b'settled and', 1)
(b"it's getting", 1)
(b'getting stale', 1)
(b'stale yeah', 1)
(b'ever leave', 1)
(b'unlocked', 1)
(b'door unlocked', 1)
(b'unlocked so', 1)
(b'just walk', 1)
(b'walk in', 1)
(b'knock', 2)
(b'they knock', 1)
(b'knock you', 1)
(b'like come', 1)
(b'seem really', 1)
(b'relaxed', 1)
(b'really relaxed', 1)
(b'relaxed why', 1)
(b'there come', 1)
(b'on babe', 1)
(b'babe come', 1)
(b'in well', 1)
(b'thing yeah', 2)
(b'nonchalantly', 1)
(b'be nonchalantly', 1)
(b'nonchalantly just', 1)
(b'just yeah', 1)
(b'uh yeah', 3)
(b'derivative', 1)
(b'a derivative', 1)
(b'de

(b'your boy', 1)
(b'boy did', 1)
(b"did you're", 1)
(b'be spending', 1)
(b'spending the', 1)
(b'entire night', 1)
(b'night most', 1)
(b'most likely', 1)
(b'likely with', 1)
(b'person why', 1)
(b'why not', 2)
(b'not show', 1)
(b'show a', 1)
(b'respect', 3)
(b'of respect', 1)
(b'respect to', 1)
(b'driving', 4)
(b"who's driving", 1)
(b'driving you', 1)
(b'destination', 1)
(b'the destination', 1)
(b'destination no', 1)
(b'the 15', 1)
(b"minutes you're", 1)
(b'car how', 1)
(b'how how', 1)
(b"about I've", 1)
(b'spend a', 1)
(b'whole the', 1)
(b'the diet', 1)
(b'diet with', 1)
(b'that friend', 1)
(b"friend can't", 1)
(b"can't wait", 1)
(b'wait to', 1)
(b'catch', 4)
(b'to catch', 2)
(b'catch up', 1)
(b'up them', 1)
(b'talk to', 5)
(b'who I', 1)
(b'have nothing', 1)
(b'nothing in', 1)
(b'in common', 1)
(b"common that's", 1)
(b'avoiding', 1)
(b"that's avoiding", 1)
(b'avoiding Christians', 1)
(b'Christians question', 1)
(b'question how', 2)
(b'driver just', 1)
(b'by paying', 1)
(b'paying him', 1

(b'with really', 1)
(b'Goods and', 2)
(b'also of', 1)
(b'course our', 1)
(b'our patches', 1)
(b'patches so', 1)
(b'so see', 1)
(b'minutes from', 2)
(b'from now', 1)
(b'now yeah', 1)
(b'yeah yep', 1)
(b'yep really', 1)
(b'Xin', 1)
(b'good Xin', 1)
(b'Xin really', 1)
(b'good you', 1)
(b'the bump', 2)
(b'bump really', 2)
(b'brilliantly', 1)
(b'a brilliantly', 1)
(b'brilliantly bump', 1)
(b'bump look', 1)
(b"look we've", 1)
(b"we've always", 1)
(b'always loved', 1)
(b'loved hearing', 1)
(b'and because', 1)
(b"because we're", 2)
(b'live-streaming', 1)
(b"we're live-streaming", 1)
(b'live-streaming our', 1)
(b'episodes on', 1)
(b'live you', 1)
(b'submit', 1)
(b'can submit', 1)
(b'submit them', 1)
(b'them at', 2)
(b'big screen', 1)
(b'screen so', 1)
(b'so do', 2)
(b'Katie', 1)
(b'what Katie', 1)
(b'Halladay', 1)
(b'Katie Halladay', 1)
(b'Halladay thinks', 1)
(b'cool when', 1)
(b'shake', 1)
(b'you shake', 1)
(b'shake your', 1)
(b'duvet', 2)
(b'your duvet', 1)
(b'duvet into', 1)
(b'the duvet', 

(b'Jesus', 1)
(b'exclusively Jesus', 1)
(b'Jesus hot', 1)
(b'hot today', 1)
(b"today I'll", 1)
(b"I'll open", 1)
(b'aah', 1)
(b'fridge aah', 1)
(b'aah how', 1)
(b'how else', 1)
(b'get cold', 1)
(b'cold fridge', 1)
(b'fridge risk', 1)
(b'risk are', 1)
(b'are because', 1)
(b'will I', 1)
(b'really heavily', 1)
(b'heavily err', 1)
(b'fridge close', 1)
(b'close as', 1)
(b'feel guilty', 1)
(b'guilty when', 1)
(b'peek', 1)
(b'I peek', 1)
(b'peek in', 1)
(b'for too', 3)
(b'long if', 1)
(b'might looking', 1)
(b'some white', 1)
(b'white know', 1)
(b'a my', 1)
(b'understand is', 1)
(b'take such', 1)
(b'a firm', 1)
(b'firm line', 1)
(b'line on', 1)
(b'this without', 1)
(b'without having', 1)
(b'concern of', 1)
(b'being worried', 1)
(b'every other', 1)
(b'other part', 1)
(b'part in', 1)
(b'conserving', 1)
(b'not conserving', 1)
(b'conserving energy', 1)
(b'energy yeah', 1)
(b'better to', 1)
(b'take what', 1)
(b'get right', 1)
(b"that's if", 1)
(b'do to', 1)
(b'my green', 1)
(b'green step', 1)
(b'st

(b"I'm Josh", 1)
(b'Josh maybe', 1)
(b"I'll throw", 1)
(b'to sort', 3)
(b'of explain', 1)
(b'explain why', 1)
(b"why we've", 1)
(b"we've taken", 1)
(b'taken this', 1)
(b'this course', 1)
(b'course of', 1)
(b'action', 1)
(b'of action', 1)
(b'action yeah', 1)
(b"break it's", 1)
(b'just as', 1)
(b'sure a', 1)
(b'experiencing', 1)
(b'are experiencing', 1)
(b'experiencing here', 1)
(b'Australia', 1)
(b'in Australia', 1)
(b'Cove', 1)
(b'Australia Cove', 1)
(b"Cove it's", 1)
(b'been pretty', 1)
(b'pretty tough', 1)
(b'tough for', 1)
(b'me personally', 1)
(b"personally I've", 1)
(b'struggled', 1)
(b"I've struggled", 1)
(b'struggled a', 1)
(b'bit with', 1)
(b'supportive', 1)
(b'really supportive', 1)
(b'supportive and', 1)
(b'and helping', 1)
(b'me through', 1)
(b'difficult but', 1)
(b'struggling', 1)
(b'are struggling', 1)
(b'struggling make', 1)
(b'you reach', 1)
(b'reach out', 1)
(b'break just', 1)
(b'a hold', 1)
(b'everything again', 1)
(b'love doing', 1)
(b'and really', 1)
(b'producing', 1

In [163]:
trigram_sentences_project = []
documents = p69
sentence_stream = [doc.split(" ") for doc in documents]

bigram = Phrases(sentence_stream, min_count=25, delimiter=b' ')
trigram = Phrases(bigram[sentence_stream], min_count=10, delimiter=b' ')

for sent in sentence_stream:
    #bigrams_ = [b for b in bigram[sent] if b.count(' ') == 1]
    #trigrams_ = [t for t in trigram[bigram[sent]] if t.count(' ') == 2]
    bigrams_ = bigram[sent]
    trigrams_ = trigram[bigram[sent]]
    trigram_sentences_project.append(trigrams_)

INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:PROGRESS: at sentence #10000, processed 12019 words and 49 word types
INFO:gensim.models.phrases:PROGRESS: at sentence #20000, processed 24018 words and 51 word types
INFO:gensim.models.phrases:PROGRESS: at sentence #30000, processed 35985 words and 61 word types
INFO:gensim.models.phrases:PROGRESS: at sentence #40000, processed 47976 words and 62 word types
INFO:gensim.models.phrases:PROGRESS: at sentence #50000, processed 59947 words and 62 word types
INFO:gensim.models.phrases:collected 63 word types from a corpus of 70019 words (unigram + bigrams) and 58425 sentences
INFO:gensim.models.phrases:using 63 counts as vocab in Phrases<0 vocab, min_count=25, threshold=10.0, max_vocab_size=40000000>
INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #

In [93]:
patches = [patch.lower().split(" ") for patch in all_patches]

In [26]:
# https://stackoverflow.com/questions/46148182/issues-in-getting-trigrams-using-gensim/46153316

#5,2  picks up problem no problem

min_bi = 12
min_tri = 5

from gensim.models import Phrases

bigram = Phrases(patches, min_count=min_bi, delimiter=b' ',threshold=16)
trigram = Phrases(bigram[patches], min_count=min_tri, delimiter=b' ')

for sent in patches:
    bigrams_ = [b for b in bigram[sent] if b.count(' ') == 1]
    trigrams_ = [t for t in trigram[bigram[sent]] if t.count(' ') == 2]
    print(bigrams_)
    print(trigrams_)

INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:collected 28531 word types from a corpus of 47340 words (unigram + bigrams) and 6 sentences
INFO:gensim.models.phrases:using 28531 counts as vocab in Phrases<0 vocab, min_count=12, threshold=16, max_vocab_size=40000000>
INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:collected 28781 word types from a corpus of 46569 words (unigram + bigrams) and 6 sentences
INFO:gensim.models.phrases:using 28781 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


['talk about', 'talk about', 'talk about', "don't know", 'all right', "don't know", 'as well', 'all right', "we've got", "don't know", 'sort of', "don't know", 'kind of', "we've got", "we'll be", 'lot of', 'little bit', "we'll be", "i'm gonna", "i've got", 'kind of', 'little bit', 'make sure', 'kind of', 'little bit', 'little bit', "you've got", 'kind of', "i've got", 'as well', 'lot of', 'all right', 'sort of', 'talking about', 'kind of', 'as well', "don't know", 'kind of', 'kind of', "you've got", "you've got", "we've got", "we've got", "you've got", 'kind of', 'all right', 'kind of', "don't know", 'lot of', 'kind of', "i'm gonna", "i'm gonna", 'lot of', 'kind of', 'kind of', 'make sure', "don't know", "don't know", "you've got", 'lot of', 'lot of', 'talking about', "i've got", "don't know", "i'm gonna", "i'm gonna", "i'm gonna", 'make sure', 'all right', 'oh wow', 'talking about', 'talking about', "you've got", "i've got", "don't know", "don't know", 'kind of', 'talking about', 'tal

['really good', 'really good', 'talk about', "don't know", "don't know", 'big prize', 'big prize', 'big prize', 'big prize', 'big prize', "i'm gonna", 'as well', 'big prize', 'big prize', 'hang on', 'little bit', 'sort of', 'as well', "we'll be", "i've got", 'tea towel', 'tea towel', 'tea towel', 'tea towel', 'tea towel', 'tea towel', 'tea towel', 'little bit', 'tea towel', 'make sure', "don't know", 'tea towel', "don't know", 'tea towel', 'tea towel', 'tea towel', 'as well', "you've got", 'tea towel', 'tea towel', "don't know", 'tea towel', 'talking about', 'talking about', 'hang on', "don't know", "i've got", 'youtube live', "you've got", "i've got", "we've got", 'lot of', 'lot of', 'talk about', 'lot of', "i've been", 'as well', 'sort of', "you've got", "i'm gonna", "you've got", 'lot of', 'kind of', 'kind of', 'kind of', 'kind of', 'hang on', 'this week', "i'm gonna", 'as well', 'talking about', "don't know", "i'm gonna", "i'm gonna", "i'm gonna", "you've got", "you've got", "i'm g

In [345]:
#id2word.filter_extremes(no_below=1)
#id2word.save_as_text('dictionary_filtered_extremes.txt',sort_by_word=False)
#print(len(id2word))

#id2word.filter_extremes(no_below=1,no_above=0.9)
#id2word_clean = corpora.Dictionary(data_lemmatized).filter_extremes(no_below=5, no_above=0.5, keep_n=1)
#id2word_2 = corpora.Dictionary(data_lemmatised)